In [1]:

import os
from CNN import SequenceCNN
from torch.utils.data import DataLoader as TorchDataLoader
from tqdm import tqdm

In [2]:
import torch
from torch.optim import lr_scheduler

import torch.optim as optim
import torch.nn as nn
from TemperatureDataset import TemperatureDataset
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms

In [3]:
dataset = TemperatureDataset('data_temp_max', 'data_temp_min')

100%|██████████| 562/562 [00:44<00:00, 12.72it/s]


In [4]:
# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each set
batch_size = 32  # You can adjust this as needed

train_loader = TorchDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = TorchDataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = TorchDataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
best_params = {
    'hidden_dims': [32, 64],
    'kernel_sizes': [5, 5],
    'lr':  0.002507800985448251,
    'batch_size': 32,
    'activations': ['leaky_relu', 'relu'],
}

In [ ]:
activations_map = {
    'relu': torch.nn.ReLU,
    'tanh': torch.nn.Tanh,
    'sigmoid': torch.nn.Sigmoid,
    'leaky_relu': torch.nn.LeakyReLU,
    'elu': torch.nn.ELU,
    'gelu': torch.nn.GELU,
    'selu': torch.nn.SELU,
    'none': nn.Identity
}
# Prepare activations

# Build and train the best model on the full training set
sample_input, sample_target = dataset[0]
input_dim = sample_input.shape[-1]
output_dim = sample_target.shape[-1]
best_model = SequenceCNN(
    input_dim=input_dim,
    hidden_dims=list(best_params['hidden_dims']),
    kernel_sizes=list(best_params['kernel_sizes']),
    output_dim=output_dim,
    activations=best_params['activations'],
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

optimizer = optim.Adam(best_model.parameters(), lr=best_params['lr'])
criterion = nn.MSELoss()

# Use the full train_loader for training
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(num_epochs):
    best_model.train()
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = best_model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # Save the best model
            torch.save(best_model.state_dict(), 'best_model_temp_cnn.pt')
torch.save(best_model.state_dict(), 'last_model_temp_cnn.pt')
# Evaluate on the test set
best_model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test loss: {test_loss}")

Training Epoch 48/50:   0%|          | 378/80522 [00:01<04:26, 301.05it/s]

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score

# Collect all predictions and targets from the test set
all_preds = []
all_targets = []
best_model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        all_preds.append(outputs.cpu().numpy())
        all_targets.append(targets.cpu().numpy())
all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)
# Calculate regression metrics
mse = mean_squared_error(all_targets, all_preds)
mae = mean_absolute_error(all_targets, all_preds)
r2 = r2_score(all_targets, all_preds)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

# For accuracy and F1, binarize or discretize if appropriate
# Example: classify if temperature > threshold (e.g., median)
threshold = np.median(all_targets)
y_true = (all_targets > threshold).astype(int)
y_pred = (all_preds > threshold).astype(int)

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
